<a href="https://colab.research.google.com/github/smduarte/spbd-2425/blob/main/docs/labs/lab3/SPBD_Labs_mapreduce2_exercise_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python MapReduce Exercises

##1. MrJob MapReduce Word Frequency

Using the [MrJob](https://mrjob.readthedocs.io/en/latest/) library, create a map-reduce program that counts the number of occurrences of each word.

a) No sorting required, words and frequency can appear in any order;

b) Sorted by word, in increasing alphabetical order;

c) Sorted by frequency (the words with higher occurrence first).

Check the MrJob documentation to see how multi-step MapReduce jobs can
be implemented in the same Python class.

In [ ]:
#@title Download the dataset and install MrJob
!wget -q -O os_maias.txt https://www.dropbox.com/s/n24v0z7y79np319/os_maias.txt?dl=0
!pip install mrjob --quiet
!wget -q -O /etc/mrjob.conf https://raw.githubusercontent.com/smduarte/spbd-2425/main/lab2/mrjob.conf

In [ ]:
#@title 1a)
%%file word_freq_1a.py

import string
from mrjob.job import MRJob, MRStep

class MRWordCountFrequency1a(MRJob):

    def mapper(self, _, line):
      line = line.strip()
      line = line.translate(str.maketrans('', '', string.punctuation+'«»'))
      for word in line.split():
        yield word, 1

    def reducer(self, key, values):
        yield key, sum(values)

if __name__ == '__main__':
    MRWordCountFrequency1a.run()

In [ ]:
!rm -rf results
!python -m word_freq_1a  --output-dir results --cleanup NONE os_maias.txt
!cat results/*

##1c MrJob MapReduce Word Frequency

In [ ]:
%%file desc_word_freq.py

import string
from mrjob.job import MRJob, MRStep

MAX_FREQ=100000

class MRWordCountFrequency(MRJob):

    def mapper_words(self, _, line):
      line = line.strip()
      line = line.translate(str.maketrans('', '', string.punctuation+'«»'))
      for word in line.split():
        yield word, 1

    def reducer_words(self, key, values):
        yield key, sum(values)

    def mapper_partition_sort(self, word, freq):
      yield "{:05d}".format(MAX_FREQ - freq), word

    def reducer_partition_sort(self, freq, words):
      for word in words:
        yield word, MAX_FREQ-int(freq)

    def steps(self):
        return [MRStep(mapper=self.mapper_words, reducer=self.reducer_words),
                 MRStep(mapper=self.mapper_partition_sort, reducer=self.reducer_partition_sort)]

if __name__ == '__main__':
    MRWordCountFrequency.run()

In [ ]:
!rm -rf results
!python -m desc_word_freq  --output-dir results --cleanup NONE os_maias.txt
!head results/*

##2. Weblog DDOS Attack Analysis

In [31]:
!wget -q -O web.log https://www.dropbox.com/s/0r8902uj9yum7dg/web.log?dl=0
!wc web.log

  143457   860742 11758533 web.log


### 1. Count the number of unique IP addresses involved in the DDOS attack.



In [32]:
%%file unique_ips.py

from mrjob.job import MRJob, MRStep

class MRUniqueIPs(MRJob):

    def steps(self):
      return [MRStep(mapper=self.mapper_ip, reducer=self.reducer_ip),
              MRStep(reducer=self.reducer_filter)]

    def mapper_ip(self, _, line):
      _, ip_source, _ = line.strip().split(' ', 2)
      yield ip_source, None

    def reducer_ip(self, ip_source, _):
      yield None, 1

    def reducer_filter(self, _, values):
      yield "UNIQUE IPs", sum(values)

if __name__ == '__main__':
    MRUniqueIPs.run()

Writing unique_ips.py


In [33]:
!rm -rf results
!python -m unique_ips  --output-dir results --cleanup NONE web.log
!head results/*

Using configs in /etc/mrjob.conf
No configs specified for inline runner
Running step 1 of 2...
Creating temp directory /tmp/unique_ips.root.20241002.144243.886785
Running step 2 of 2...
job output is in results
"UNIQUE IPs"	167


### 2. For each interval of 10 seconds, provide the following information: [number of requests, average execution time, maximum time, minimum time]





In [36]:
%%file interval_stats.py

from statistics import *
from mrjob.job import MRJob, MRStep

class MRIntervalStats(MRJob):

  def mapper(self, _, line):
        vals = line.strip().split(' ')
        timestamp = vals[0]
        execution_time = float(vals[5])
        interval = timestamp[0:18] # YYYY-MM-DDTHH:MM:S -> 10s intervals
        yield interval, execution_time

  def reducer(self, interval, values):
      times = list(values)
      yield interval, (len(times), min(times), mean(times), max(times))

if __name__ == '__main__':
    MRIntervalStats.run()

Overwriting interval_stats.py


In [37]:
%%shell
rm -rf results
python -m interval_stats --output-dir results --cleanup NONE web.log && head results/*

Using configs in /etc/mrjob.conf
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/interval_stats.root.20241002.144410.819408
job output is in results
==> results/part-00000 <==
"2016-12-06T08:58:3"	[483, 0.013, 7.593424430641822, 46.849]
"2016-12-06T08:58:4"	[2611, 0.014, 30.15984565300651, 69.654]
"2016-12-06T08:58:5"	[5500, 0.017, 38.52511163636364, 80.846]
"2016-12-06T08:59:0"	[6914, 0.018, 38.534382123228234, 81.659]
"2016-12-06T08:59:1"	[6271, 0.017, 32.96384978472333, 83.993]
"2016-12-06T08:59:2"	[5434, 0.051, 17.29333143172617, 77.967]
"2016-12-06T08:59:3"	[8015, 0.056, 11.21015221459763, 67.441]
"2016-12-06T08:59:4"	[7947, 0.914, 7.7618157795394485, 65.706]

==> results/part-00001 <==
"2016-12-06T08:59:5"	[5983, 0.678, 3.8216643824168477, 54.29]
"2016-12-06T09:00:0"	[6882, 0.017, 8.649971519907004, 45.314]
"2016-12-06T09:00:1"	[9719, 0.225, 7.857372672085606, 34.406]
"2016-12-06T09:00:2"	[6616, 0.014, 4.611345223700121, 25.847]
"2016-12

### 3. Create an inverted index that, for each interval of 10 seconds, has a list of (unique) IPs executing accesses (to each URL).

In [38]:
%%file inverted_index.py

from mrjob.job import MRJob, MRStep

class MRInvertedIndex(MRJob):

  def mapper(self, _, line):
        vals = line.strip().split(' ')
        if len(vals) >= 6:
          timestamp = vals[0]
          interval = timestamp[0:18] # YYYY-MM-DDTHH:MM:S -> 10s intervals

          source_ip = vals[1]
          target_url = vals[4]
          yield "{}-{}".format(interval, target_url), source_ip

  def reducer(self, key, values):
    yield key, list(values)

if __name__ == '__main__':
    MRInvertedIndex.run()

Writing inverted_index.py


In [39]:
%%shell
rm -rf results
python -m inverted_index --output-dir results web.log && head results/*

Using configs in /etc/mrjob.conf
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/inverted_index.root.20241002.144508.154605
job output is in results
Removing temp directory /tmp/inverted_index.root.20241002.144508.154605...
==> results/part-00000 <==
"2016-12-06T08:58:3-/codemove/01IX95N3AFP4"	["120.52.73.98", "120.52.73.98", "120.52.73.98", "120.52.73.98", "120.52.73.98", "120.52.73.98", "120.52.73.98", "120.52.73.98", "120.52.73.98", "120.52.73.98", "120.52.73.98"]
"2016-12-06T08:58:3-/codemove/0GLNQSHCISWJ"	["120.52.73.98", "120.52.73.98", "120.52.73.98", "120.52.73.98", "120.52.73.98", "120.52.73.98", "120.52.73.98", "120.52.73.98", "120.52.73.98", "120.52.73.98", "120.52.73.98"]
"2016-12-06T08:58:3-/codemove/1N80W0N2R36C"	["120.52.73.97", "120.52.73.97", "120.52.73.97", "120.52.73.97", "120.52.73.97", "120.52.73.97", "120.52.73.97", "120.52.73.97", "120.52.73.97", "120.52.73.97", "120.52.73.97"]
"2016-12-06T08:58:3-/codemove/1U6HCG3V2S9D"